In [201]:
import numpy as np
import netCDF4 as nc
import auxiliary as au
import regression as reg
import datetime as dt
from matplotlib import pyplot as plt
from scipy import io
from scipy.interpolate import interp2d
import os
import sys
from scipy.interpolate import griddata
import h5py
import time
import random
import datetime

def divide_train_test(data, dividenum, randseed=-1):
    if randseed == -1:
        random.seed(time.time())
    num = len(data)
    subnum = int(num / dividenum)
    data_train = np.zeros([dividenum, num - subnum],dtype=int)
    data_test = np.zeros([dividenum, subnum],dtype=int)
    randindex = random.sample(range(num), num)
    for i in range(dividenum):
        data_test[i, :] = np.sort(data[randindex[i * subnum:(i + 1) * subnum]])
        data_train[i, :] = np.setdiff1d(data, data_test[i])
    return data_train, data_test


def double_cvindex(gmet_stndatafile, dividenum):
    # index for double cross-validation
    datatemp = np.load(gmet_stndatafile)
    prcp_stn0 = datatemp['prcp_stn'][:, 0]
    tmean_stn0 = datatemp['tmean_stn'][:, 0]
    prcp_stnindex = np.argwhere(~np.isnan(prcp_stn0))
    prcp_stnindex = prcp_stnindex.flatten()
    tmean_stnindex = np.argwhere(~np.isnan(tmean_stn0))
    tmean_stnindex = tmean_stnindex.flatten()

    subnum1 = int(len(prcp_stnindex) / dividenum)
    subnum2 = int((len(prcp_stnindex) - subnum1) / dividenum)
    # prcp_testindex1 = np.zeros([dividenum,subnum1])
    # prcp_trainindex1 = np.zeros([dividenum,len(prcp_stnindex) - subnum1])
    prcp_trainindex1, prcp_testindex1 = divide_train_test(prcp_stnindex, dividenum, randseed=123)
    prcp_testindex2 = np.zeros([dividenum, dividenum, subnum2], dtype=int)
    prcp_trainindex2 = np.zeros([dividenum, dividenum, len(prcp_stnindex) - subnum1 - subnum2], dtype=int)
    for i in range(dividenum):
        traini, testi = divide_train_test(prcp_trainindex1[i, :], dividenum, randseed=123)
        prcp_trainindex2[i, :, :] = traini
        prcp_testindex2[i, :, :] = testi

    subnum1 = int(len(tmean_stnindex) / dividenum)
    subnum2 = int((len(tmean_stnindex) - subnum1) / dividenum)
    # tmean_testindex1 = np.zeros([dividenum,subnum1])
    # tmean_trainindex1 = np.zeros([dividenum,len(tmean_stnindex) - subnum1])
    tmean_trainindex1, tmean_testindex1 = divide_train_test(tmean_stnindex, dividenum, randseed=123)
    tmean_testindex2 = np.zeros([dividenum, dividenum, subnum2], dtype=int)
    tmean_trainindex2 = np.zeros([dividenum, dividenum, len(tmean_stnindex) - subnum1 - subnum2], dtype=int)
    for i in range(dividenum):
        traini, testi = divide_train_test(tmean_trainindex1[i, :], dividenum, randseed=123)
        tmean_trainindex2[i, :, :] = traini
        tmean_testindex2[i, :, :] = testi
    return prcp_trainindex1, prcp_testindex1, prcp_trainindex2, prcp_testindex2, \
           tmean_trainindex1, tmean_testindex1, tmean_trainindex2, tmean_testindex2

def calculate_anomaly(datatar, dataref, hwsize, amode, upbound=5, lowbound=0.2):
    # datatar, dataref: 2D [nstn, ntime]
    # amode: anomaly mode ('ratio' or 'diff')
    # hwsize: define time window (2*hwsize+1) used to calculate ratio (as ratio for a specific day is too variable)
    # upbound/lowbound: upper and lower limitation of ratio/difference
    if np.ndim(datatar) == 1: # only one time step
        datatar = datatar[:,np.newaxis]
        dataref = dataref[:,np.newaxis]

    nstn, ntime = np.shape(datatar)
    if ntime < hwsize * 2 + 1:
        print('The window size is larger than time steps when calculating ratio between tar and ref datasets')
        print('Please set a smaller hwsize')
        sys.exit()

    indnan = np.isnan(datatar) | np.isnan(dataref)
    datatar[indnan] = np.nan
    dataref[indnan] = np.nan
    del indnan

    anom = np.ones([nstn, ntime])

    for i in range(ntime):
        if i < hwsize:
            windex = np.arange(hwsize * 2 + 1)
        elif i >= ntime - hwsize:
            windex = np.arange(ntime - hwsize * 2 - 1, ntime)
        else:
            windex = np.arange(i - hwsize, i + hwsize + 1)
        dtari = np.nanmean(datatar[:, windex], axis=1)
        drefi = np.nanmean(dataref[:, windex], axis=1)

        if amode == 'ratio':
            temp = drefi / dtari
            temp[(dtari == 0) & (drefi == 0)] = 1
            anom[:, i] = temp
        elif amode == 'diff':
            anom[:, i] = drefi - dtari
        else:
            sys.exit('Unknow amode. Please use either ratio or diff')

    anom[anom > upbound] = upbound
    anom[anom < lowbound] = lowbound
    return anom


def extrapolation(latin, lonin, datain, latout, lonout, nearnum):
    # datain: one or multiple time steps
    wexp = 3
    if np.ndim(datain) == 1:  # add time axis
        datain = datain[:, np.newaxis]
    latin[np.isnan(datain[:, 0])] = np.nan
    lonin[np.isnan(datain[:, 0])] = np.nan

    if np.ndim(latout) == 1: # extrapolate to station points
        nearstn_loc, nearstn_dist = findnearstn(latin, lonin, latout, lonout, nearnum, 1)
        num = len(latout)
        ntimes = np.shape(datain)[1]
        dataout = np.zeros([num, ntimes])
        for i in range(num):
            if np.mod(i,500) == 0:
                print(i)
            dataini = datain[nearstn_loc[i, :], :]
            disti = nearstn_dist[i, :]
            weighti = au.distanceweight(disti, np.max(disti) + 1, wexp)
            weighti = weighti / np.sum(weighti)
            for j in range(ntimes):
                dataout[i, j] = np.sum(dataini[:, j] * weighti)

    elif np.ndim(latout) == 2: # extrapolate to gridds
        nearstn_loc, nearstn_dist = findnearstn(latin, lonin, latout, lonout, nearnum, 0)
        nrows, ncols, ntimes = np.shape(datain)
        dataout = np.zeros([nrows, ncols, ntimes])
        for r in range(nrows):
            for c in range(ncols):
                dataini = datain[nearstn_loc[r, c, :], :]
                disti = nearstn_dist[r, c, :]
                weighti = au.distanceweight(disti, np.max(disti) + 1, wexp)
                weighti = weighti / np.sum(weighti)
                for j in range(ntimes):
                    dataout[r, c, j] = np.sum(dataini[:, j] * weighti)
    else:
        print('The dimensions of tarlat or tarlon are larger than 2')
        sys.exit()

    return dataout


def findnearstn(stnlat, stnlon, tarlat, tarlon, nearnum, noself):
    # only use lat/lon to find near stations without considering distance in km
    # stnlat/stnlon: 1D
    # tarlat/tarlon: 1D or 2D
    # noself: 1--stnlat and tarlat have overlapped stations, which should be excluded from stnlat

    stnll = np.zeros([len(stnlat), 2])
    stnll[:, 0] = stnlat
    stnll[:, 1] = stnlon

    if len(np.shape(tarlat)) == 1:
        num = len(tarlat)
        nearstn_loc = np.zeros([num, nearnum],dtype=int)
        nearstn_dist = np.zeros([num, nearnum],dtype=float)
        for i in range(num):
            tari = np.array([tarlat[i], tarlon[i]])
            dist = au.distance(tari, stnll)
            dist[np.isnan(dist)] = 1000000000
            if noself == 1:
                dist[dist == 0] = np.inf  # may not be perfect, but work for SCDNA
            indi = np.argsort(dist)
            nearstn_loc[i, :] = indi[0:nearnum]
            nearstn_dist[i, :] = dist[nearstn_loc[i, :]]
    elif len(np.shape(tarlat)) == 2:
        nrows, ncols = np.shape(tarlat)
        nearstn_loc = np.zeros([nrows, ncols, nearnum],dtype=int)
        nearstn_dist = np.zeros([nrows, ncols, nearnum],dtype=float)
        for r in range(nrows):
            for c in range(ncols):
                tari = np.array([tarlat[r, c], tarlon[r, c]])
                dist = au.distance(tari, stnll)
                dist[np.isnan(dist)] = 1000000000
                indi = np.argsort(dist)
                nearstn_loc[r, c, :] = indi[0:nearnum]
                nearstn_dist[r, c, :] = dist[nearstn_loc[r, c, :]]
    else:
        print('The dimensions of tarlat or tarlon are larger than 2')
        sys.exit()

    return nearstn_loc, nearstn_dist


def error_correction(dataori, anomaly, mode='ratio'):
    # default: time is the last dimension
    if mode == 'ratio':
        datacorr = dataori * anomaly
    elif mode == 'diff':
        datacorr = dataori + anomaly
    else:
        sys.exit('Wrong error correction mode')
    return datacorr


def calweight(obsall, preall, mode='RMSE', preprocess=True):
    nstn, ntime = np.shape(obsall)
    met = np.zeros(nstn)
    for i in range(nstn):
        obs = obsall[i, :]
        pre = preall[i, :]
        if preprocess:
            # delete the nan values
            ind_nan = np.isnan(obs) | np.isnan(pre)
            obs = obs[~ind_nan]
            pre = pre[~ind_nan]
        if mode == 'RMSE':
            met[i] = np.sqrt(np.sum(np.square(obs - pre)) / len(obs))  # RMSE
        elif mode == 'CC':
            temp = np.corrcoef(obs, pre)
            met[i] = temp[0][1]  # CC
        else:
            sys.exit('Unknown inputs for calmetric')

    if mode == 'RMSE':
        weight = 1 / (met ** 2)
    elif mode == 'CC':
        met[met < 0] = 0
        weight = met ** 2
    else:
        sys.exit('Unknown inputs for calmetric')

    return weight


def calrmse(dtar, dref):
    if np.ndim(dtar) == 1:
        dtar = dtar[np.newaxis, :]
        dref = dref[np.newaxis, :]
    nstn, ntimes = np.shape(dtar)
    rmse = np.nan * np.zeros(nstn)
    for i in range(nstn):
        rmse[i] = np.sqrt(np.nansum(np.square(dtar[i, :] - dref[i, :])) / ntimes)  # RMSE

    return rmse


def ismember(a, b):
    # tf = np.in1d(a,b) # for newer versions of numpy
    tf = np.array([i in b for i in a])
    u = np.unique(a[tf])
    index = np.array([(np.where(b == i))[0][-1] if t else 0 for i, t in zip(a, tf)])
    return tf, index


def weightmerge(data, weight):
    if np.ndim(data) == 2:
        num, nmodel = np.shape(data)
        dataout = np.zeros(num)
        for i in range(num):
            dataout[i] = np.sum(data[i, :] * weight[i, :]) / np.sum(weight[i, :])
    elif np.ndim(data) == 3:
        nrows, ncols, nmodel = np.shape(data)
        dataout = np.zeros([nrows, ncols])
        for r in range(nrows):
            for c in range(ncols):
                if not np.isnan(data[r, c, 0]):
                    dataout[r, c] = np.sum(data[r, c, :] * weight[r, c, :]) / np.sum(weight[r, c, :])
    return dataout

def m_DateList(year_start, year_end, mode):
    # generate a date list (yyyymmdd) between start year and end year
    # mode: 'ByDay', 'ByMonth', 'ByYear': time scales of input files
    date_start = datetime.date(year_start, 1, 1)
    date_end = datetime.date(year_end, 12, 31)
    daynum = (date_end - date_start).days + 1

    # generate date in format: yyyymmdd
    date_ymd = np.zeros(daynum, dtype=int)
    dated = date_start
    for d in range(daynum):
        if d > 0:
            dated = dated + datetime.timedelta(days=1)
        date_ymd[d] = int(dated.strftime("%Y%m%d"))
    date_number = {'yyyymmdd': date_ymd,
                   'yyyymm': np.floor(date_ymd / 100).astype(int),
                   'yyyy': np.floor(date_ymd / 10000).astype(int),
                   'mm': np.floor(np.mod(date_ymd, 10000) / 100).astype(int),
                   'dd': np.mod(date_ymd, 100).astype(int)}

    # generate file list
    if mode == 'ByDay':
        datemode = date_number['yyyymmdd']
    else:
        if mode == 'ByMonth':
            datemode = date_number['yyyymm']
        elif mode == 'ByYear':
            datemode = date_number['yyyy']
        datemode = np.unique(datemode)

    date_list = [' '] * len(datemode)
    for i in range(len(datemode)):
        date_list[i] = str(datemode[i])

    return date_list, date_number

########################################################################################################################

# basic settings
lontar = np.arange(-180 + 0.05, -50, 0.1)
lattar = np.arange(85 - 0.05, 5, -0.1)
var = 'prcp'   # ['prcp', 'tmean', 'trange']: this should be input from sbtach script
corrmode = 'ratio'  # ratio or diff: mode for error correction
hwsize = 15  # define time window (2*hwsize+1) used to calculate ratio (as ratio for a specific day is too variable)
nearnum = 8  # the number of nearby stations used to extrapolate points to grids (for correction and merging)
weightmode = 'RMSE'  # the metric used to guide merging (CC or RMSE). Weight = CC**2 or 1/RMSE**2
dividenum = 10  # divide the datasets into X parts, e.g. 10-fold cross-validation
anombound = [0.2, 5] # upper and lower bound when calculating the anomaly between target and reference data for correction

if corrmode == 'diff':
    # default settings in this study since diff is for tmean and trange
    hwsize = 0
    anombound = [-999, 999]

# input files
# station list and data
# gmet_stnfile = '/home/gut428/GMET/eCAI_EMDNA/StnGridInfo/stnlist_whole.txt'
# gmet_stndatafile = '/home/gut428/stndata_whole.npz'
gmet_stnfile = '/Users/localuser/GMET/pyGMET_NA/stnlist_whole.txt'
gmet_stndatafile = '/Users/localuser/GMET/pyGMET_NA/stndata_whole.npz' # to be saved. only process when absent

# downscaled reanalysis data at station points
# file_readownstn = ['/ERA5_downto_stn.npz',
#                    '/MERRA2_downto_stn.npz',
#                    '/JRA55_downto_stn.npz']
file_readownstn = ['/Users/localuser/Research/Test/ERA5_downto_stn.npz',
                  '/Users/localuser/Research/Test/MERRA2_downto_stn.npz',
                  '/Users/localuser/Research/Test/JRA55_downto_stn.npz']

# mask file
# file_mask = '/datastore/GLOBALWATER/CommonData/EMDNA/DEM/NA_DEM_010deg_trim.mat'
file_mask = './DEM/NA_DEM_010deg_trim.mat'

# downscaled reanalysis: gridded data
path_readown = ['', '', '']
prefix = ['ERA5_', 'MERAA2_', 'JRA55_']

# output files
# train and test index file
ttindexfile = '/Users/localuser/Research/Test/2layer_train_test_index.npz'

# output corrected and merged data
path_reacorr = ['', '', '']
path_merge = ''
file_error_corr = ['', '', ''] # the error at all station points for corrected reanalysis data (based on cross-validation)

########################################################################################################################

# basic processing
# mask
mask = io.loadmat(file_mask)
mask = mask['DEM']
mask[~np.isnan(mask)] = 1  # 1: valid pixels
# attributes
reanum = len(file_readownstn)
nrows, ncols = np.shape(mask)
lontarm, lattarm = np.meshgrid(lontar, lattar)

# date
date_list, date_number = m_DateList(1979, 2018, 'ByYear')

In [198]:
if not os.path.isfile(ttindexfile):
    prcp_trainindex1, prcp_testindex1, prcp_trainindex2, prcp_testindex2, \
    tmean_trainindex1, tmean_testindex1, tmean_trainindex2, tmean_testindex2 = double_cvindex(gmet_stndatafile,
                                                                                              dividenum)
    np.savez_compressed(ttindexfile, prcp_trainindex1=prcp_trainindex1, prcp_testindex1=prcp_testindex1,
                        prcp_trainindex2=prcp_trainindex2, prcp_testindex2=prcp_testindex2,
                        tmean_trainindex1=tmean_trainindex1, tmean_testindex1=tmean_testindex1,
                        tmean_trainindex2=tmean_trainindex2, tmean_testindex2=tmean_testindex2)
    del prcp_trainindex1, prcp_testindex1, prcp_trainindex2, prcp_testindex2, \
        tmean_trainindex1, tmean_testindex1, tmean_trainindex2, tmean_testindex2

taintestindex = np.load(ttindexfile)

In [202]:
# load downscaled reanalysis for all stations
readata_stn = [''] * reanum
for rr in range(reanum):
    dr = np.load(file_readownstn[rr])
    temp = dr[var + '_readown']
    if prefix[rr] != 'MERRA2_': # unify the time length of all data as MERRA2 lacks 1979
        temp = temp[:,365:]
    readata_stn[rr] = temp
    del dr

# load observations for all stations
datatemp = np.load(gmet_stndatafile)
stndata = datatemp[var + '_stn']
stndata = stndata[:, 365:]
stnlle = datatemp['stn_lle']
nstn, ntimes = np.shape(stndata)
del datatemp

# initialize
metric_merge_stn = np.nan * np.zeros(nstn) # accuracy metric of merged reanalysis at station points
error_merge_stn = np.nan * np.zeros([nstn,ntimes]) # mean error (merge minus observation)
metric_reacorr_stn = [''] * reanum # accuracy metric of corrected reanalysis at station points
error_reacorr_stn = [''] * reanum
for i in range(reanum):
    metric_reacorr_stn[i] = np.nan * np.zeros(nstn)
    error_reacorr_stn[i] = np.nan * np.zeros([nstn,ntimes])

In [205]:
    lay1=1
    # extract train and test index for layer-1
    if var == 'trange':
        vari = 'tmean'  # trange and tmean have the same index
    else:
        vari = var
    trainindex1 = taintestindex[vari + '_trainindex1'][lay1, :]
    testindex1 = taintestindex[vari + '_testindex1'][lay1, :]
    stndata_trainl1 = stndata[trainindex1, :]
    stndata_testl1 = stndata[testindex1, :]
    stnlle_trainl1 = stnlle[trainindex1, :]
    stnlle_testl1 = stnlle[testindex1, :]

    # merging weight of different reanalysis products at station points (trainindex1)
    reacorr_trainl1 = ['']*reanum
    for rr in range(reanum):
        reacorr_trainl1[rr] = np.zeros(np.shape(stndata_trainl1))

In [204]:
        lay2=3
        # extract train and test index for layer-2 (subsets of trainindex1)
        trainindex2 = taintestindex[vari + '_trainindex2'][lay1, lay2, :]
        testindex2 = taintestindex[vari + '_testindex2'][lay1, lay2, :]
        stndata_trainl2 = stndata[trainindex2, :]
        stndata_testl2 = stndata[testindex2, :]
        stnlle_trainl2 = stnlle[trainindex2, :]
        stnlle_testl2 = stnlle[testindex2, :]

In [206]:
        for rr in range(reanum):
            readata_trainl2 = readata_stn[rr][trainindex2, :]
            readata_testl2 = readata_stn[rr][testindex2, :]

            # calculate corrected reanalysis data
            # calculate anomaly at the train stations
            anom_ori = calculate_anomaly(readata_trainl2, stndata_trainl2, hwsize, corrmode,
                                         upbound=anombound[1], lowbound=[0])
            # extrapolate the ratio to the test stations (in layer-2)
            anom_ext = extrapolation(stnlle_trainl2[:, 0], stnlle_trainl2[:, 1], anom_ori,
                                      stnlle_testl2[:, 0], stnlle_testl2[:, 1], nearnum)
            # correct data at the test stations
            readata_testl2_corr = error_correction(readata_testl2, anom_ext, mode=corrmode)
            tf, index = ismember(testindex2, trainindex1)
            reacorr_trainl1[rr][index, :] = readata_testl2_corr

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:96: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:97: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:100: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:100: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:108: RuntimeWarning: invalid value encountered in greater
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:109: RuntimeWarning: invalid value encountered in less


0
500
1000
1500
2000
0
500
1000
1500
2000
0
500
1000
1500
2000


In [219]:
z=np.random.rand(10) 
np.convolve(z, np.ones((1,)) / 1, mode='same')

array([0.49492594, 0.31545068, 0.39060605, 0.12309098, 0.98420048,
       0.4857062 , 0.69419902, 0.5060986 , 0.33048425, 0.81869946])

In [299]:
file='/Users/localuser/Downloads/output_19900101-19900131.npz'
d=np.load(file)
d3=d['tmean']

In [290]:
plt.imshow(d1[:,:,0])
plt.colorbar()
plt.clim([-20,0])

SyntaxError: unexpected EOF while parsing (<ipython-input-290-9457d88f2a76>, line 4)

In [282]:
file='/Users/localuser/GMET/pyGMET_NA/stndata_whole.npz'
d=np.load(file)
d.files

['prcp_stn',
 'tmean_stn',
 'trange_stn',
 'stn_ID',
 'stn_lle',
 'stn_row',
 'stn_col']

In [298]:
print(d1[770,1000:1100,30])
print(d2[770,1000:1100,30])

[-34.24548875 -34.24572272 -34.24580942 -34.24590073 -34.24599655
 -34.24609675 -34.24620122 -34.24630984 -34.2464225  -34.24653909
 -34.24665949 -34.24678358 -34.24691125 -34.2470424  -34.24717691
 -34.24731466 -34.24745556 -34.24759948 -34.24774632 -34.24814125
 -34.24856776 -34.24899648 -34.24942729 -34.24986011 -34.25029482
 -34.25073134 -34.25116954 -34.25160933 -34.2520506  -34.25249324
 -34.25293714 -34.25338218 -34.25382827 -34.25427528 -34.2547231
 -34.25517161 -34.2556207  -34.25607025 -34.25652014 -34.25697024
 -34.25742044 -34.25787062 -34.25832065 -34.25877041 -34.25921977
 -34.25966861 -34.2601168  -34.2605642  -34.26101071 -34.26145618
 -34.26190048 -34.26234349 -34.26278507 -34.26322509 -34.26366342
 -34.26409992 -34.26453447 -34.26496692 -34.26539715 -34.26582501
 -34.26625037 -34.2666731  -34.26709306 -34.26751011 -34.26792411
 -34.26833494 -34.26874244 -34.26914648 -34.26954693 -34.26994364
 -34.27033648 -34.27072531 -34.27110998 -34.27149037 -34.27186633
 -34.272237

In [301]:
d3.dtype

dtype('float64')

In [285]:
io.savemat('test.mat',{'tm':tm})